### 📖 Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현

- 체인을 `수동으로 구현`해야 합니다.

- 체인에 `ConversationBufferMemory` 를 부여합니다.

- 해당 문서를 사용하여 RAG를 수행합니다 : https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223

- 체인에 다음 질문을 합니다:
    - Aaronson 은 유죄인가요?
    - 그가 테이블에 어떤 메시지를 썼나요?
    - Julia 는 누구인가요?

#### 💡 사용할 라이브러리 및 모듈 import 하기

In [183]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder 

from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

#### 💡 언어모델(LLM) 생성하기

In [184]:
llm = ChatOpenAI(
    temperature=0.1,        # 창의성 (0 ~ 2)
    model='gpt-3.5-turbo',  # 사용 모델 지정 (Default : gpt-3.5-turbo)
)

#### 💡 메모리 생성 

In [185]:
memory = ConversationBufferMemory(
    max_token_limit=120,
    return_messages=True,           # 문자열 기반이 아닌, ChatPromptTemplate 에서 사용할 수 있는 형태로 반환
)

#### 💡 1단계 : `문서 로드 (Load Documents)`

In [186]:
loader = UnstructuredFileLoader("./files/challenge.txt")

#### 💡 2단계 : `텍스트 분할 (Split Text)`

In [187]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",                 # 특정 기준으로 분할
    chunk_size=600,
    chunk_overlap=100,
)

split_doc = loader.load_and_split(text_splitter)

#### 💡 3단계 : `임베딩 (Embedding) 생성`

In [188]:
embeddings = OpenAIEmbeddings()

# 캐시 지정
cache_dir = LocalFileStore('./.cache/')

# 캐시를 활용한 임베딩
cacehd_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

#### 💡 4단계 : `DB 생성 및 저장`

In [189]:
vectorstore = FAISS.from_documents(documents=split_doc, embedding=cacehd_embeddings)

#### 💡 5단계 : `검색기(Retriever) 생성`

In [190]:
retriever = vectorstore.as_retriever()

#### 💡 6단계 : `프롬프트(prompt) 생성`

In [191]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
    """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved {context} to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.
    """
    ),
    MessagesPlaceholder(variable_name='history'),
    ("human", "{question}")
])

#### 💡 7단계 : `Stuff Documenet Chain 안에 Memory를 연결한 Class 정의`

In [192]:
class StuffDocumentMemoryChain:
    def __init__(self, llm, prompt, memory, retriver, input_key="question"):
        self.prompt = prompt
        self.memory = memory
        self.retriver = retriver
        self.input_key = input_key
        # RunnablePassthrough.assign(history=self.load_memory)
        self.chain = {
            'context': retriver, 
            'history': RunnablePassthrough.assign(history=self.load_memory),
        } | prompt | llm

    def load_memory(self, _):
        return self.memory.load_memory_variables({})['history']

    def invoke(self, question, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: question})
        self.memory.save_context({"input": question}, {"output": answer.content})
        
        return answer.content

#### 💡 8단계 : `체인(Chain) 생성`

In [193]:
stuff_document_memory_chain = StuffDocumentMemoryChain(llm, prompt, memory, retriever)

#### 📢 [ 질문 1 ] Aaronson 은 유죄인가요 ?

In [194]:

# [ 질문 1 ] Aaronson 은 유죄인가요 ?
stuff_document_memory_chain.invoke('Is Aaronson guilty?')

TypeError: expected string or buffer